In [2]:
# 1) Напишем функцию, которая напрямую вычисляет обратную матрицу от суммы:

import numpy as np
def woodbury(A,U,V):
    return np.linalg.inv(A + U@V)

In [34]:
# 2) Теперь напишем, функцию вычисляющую обратную матрицу от суммы по формуле (1):

import numpy as np
def fastwoodbury1(A,U,V,k):
    A_inv = np.diag(1./np.diag(A)) # Бстрое обращение диагональной матрицы (А - диоганальна по условию)
    B_inv = np.linalg.inv(np.eye(k) + V @ A_inv @ U)
    return A_inv - A_inv @ U @ B_inv @ V @ A_inv

In [35]:
# 3) Сравним какая из 2 функций быстрее

import numpy as np
import timeit
code_to_test = """
def woodbury(A,U,V):
    return np.linalg.inv(A + U@V)
""" 
elapsed_time = timeit.timeit(code_to_test, number=100)/100
print(elapsed_time)

7.5999996624887e-08


In [36]:
import numpy as np
import timeit
code_to_test = """
def fastwoodbury(A,U,V,k):
    A_inv = np.diag(1./np.diag(A))
    B_inv = np.linalg.inv(np.eye(k) + V @ A_inv @ U)
    return A_inv - A_inv @ U @ B_inv @ V @ A_inv
"""
elapsed_time = timeit.timeit(code_to_test, number=100)/100
print(elapsed_time)

7.399999958579429e-08


In [37]:
# A - диагональная матрица p*p, U - p*k, V - k*p
A_inv_diag = np.array([1, 2, 3, 4, 5])
A1 = A_inv_diag.reshape(-1,1)
A1

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [38]:
U =np.array([6,7,8,9,10,11,12,13,14,15])
U1 = U.reshape(5,2)
U1

array([[ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15]])

In [39]:
A1 * U1

array([[ 6,  7],
       [16, 18],
       [30, 33],
       [48, 52],
       [70, 75]])

In [40]:
np.diag(A_inv_diag) @ U1

array([[ 6,  7],
       [16, 18],
       [30, 33],
       [48, 52],
       [70, 75]])

In [41]:
# Вывод np.diag(A_inv_diag) @ U1 тоже самое, что A1 * U1 (только * - быстрее)
# Значит можем улучшить наш код

import numpy as np
def fastwoodbury2(A,U,V,k):
    A_inv_diag = 1./np.diag(A) # Это вектор
    B_inv = np.linalg.inv(np.eye(k) + (V * A_inv_diag) @ U)
    return np.diag(A_inv_diag) - (A_inv_diag.reshape(-1,1) * U @ B_inv @ V * A_inv_diag)

In [42]:
# Рассмотрим конкретный случай 
# 1) Быстрая функция
p = 5000
k = 100
A = np.diag(np.random.randn(p))
U = np.random.randn(p, k)
V = np.random.randn(k, p)
fastwoodbury1(A,U,V,k)

array([[-8.00442692e-01, -5.55832617e+00, -1.52802558e-01, ...,
         4.38426134e-01, -7.68311085e-02, -1.45758234e+00],
       [ 8.70594474e+00, -8.59363321e+01, -2.14181879e+00, ...,
         7.30618592e+00, -1.12193782e+00, -2.47135917e+01],
       [-3.90088659e-01,  3.76050227e+00, -1.93726115e+00, ...,
        -3.02825510e-01,  7.18600622e-02,  9.38134036e-01],
       ...,
       [-1.11071557e+00,  1.15855098e+01,  3.20634611e-01, ...,
         7.29803191e-01,  1.46720708e-01,  3.04533059e+00],
       [ 5.14731318e-02, -4.39375218e-01, -1.32573599e-02, ...,
         4.87336480e-02, -9.53917170e-01, -1.14763163e-01],
       [ 6.50479337e-01, -7.55640899e+00,  2.44165214e-03, ...,
         5.68147041e-01, -8.60027656e-02, -6.42534371e+00]])

In [43]:
# 2) Медленная функция

p = 5000
k = 100
A = np.diag(np.random.randn(p))
U = np.random.randn(p, k)
V = np.random.randn(k, p)
woodbury(A,U,V)

array([[ 7.37099213e-01,  5.44918522e-03, -5.41618278e-03, ...,
         5.26618361e-02,  2.92050338e-02,  1.09140156e-03],
       [-2.20704619e-03,  2.21429080e+00, -2.31732387e-02, ...,
        -6.22843633e-02, -2.32013991e-01, -2.45041315e-03],
       [ 1.95904338e-02, -1.13146350e-03,  1.15289123e+00, ...,
        -1.42982839e-01,  4.59596716e-03, -2.08600947e-02],
       ...,
       [ 5.04520024e-02,  1.57132907e-01,  3.02613103e-02, ...,
        -6.98944151e+00, -1.00102921e-01,  9.39060727e-03],
       [ 3.95145150e-03,  2.05990256e-01,  6.56280106e-02, ...,
         4.05252995e-01,  9.71983290e+00,  1.58205480e-02],
       [-1.85961590e-03, -7.33150732e-03, -2.47049405e-03, ...,
        -1.75956068e-02, -7.33076746e-03,  5.74534214e-01]])

In [ ]:
# fastwoodbury быстрее потому что, уменьшили кол-во операций (нам легче обращать матрицу k*k, чем p*p (p>k))